# Altimetry

Some description

In [51]:
#import libraries
import xarray as xr
import numpy as np
from datetime import datetime, timedelta
import sys
from lib import geost1D, bounds
import cartopy.crs as ccrs

## Select area and cycle

In [43]:
min_lon, max_lon = -76, -73
min_lat, max_lat = 36.5, 39.5
min_cycle, max_cycle = 157, 157

## Load the dataset

In [44]:
#some description

In [52]:
#load file
loc = "./data/SLAFILT/ctoh.sla.filt.ref.TP+J1+J2+J3.gulfstream.228.nc"
ds = xr.open_dataset(loc)
#assign coordinates
ds = ds.rename({"missions_cycles": "cycles", "cycle":"xtrack_cycle"}).set_coords(["lat", "lon", "cycles"])
#subset by set bounds
point_idx, cycle_idx = bounds(ds.lat.data, 
                              ds.lon.data, 
                              ds.xtrack_cycle.data, 
                              min_lat, 
                              max_lat, 
                              min_lon, 
                              max_lon, 
                              min_cycle, 
                              max_cycle)
ds = ds.isel(nbpoints = point_idx, nbcycles = cycle_idx)
cycle_array = ds.xtrack_cycle.data
#converting JulianTime to datetime (TBC)
#print(ds['time'])
#dt_idx = ds.index["time"].to_datetimeindex()
#ds['time'] = dt_idx
ds

<xarray.Dataset>
Dimensions:               (nbpoints: 49, nbcycles: 1)
Coordinates:
    lon                   (nbpoints) float32 -73.33 -73.36 ... -74.86 -74.88
    lat                   (nbpoints) float32 36.54 36.58 36.63 ... 38.85 38.89
    cycles                (nbcycles) float32 ...
Dimensions without coordinates: nbpoints, nbcycles
Data variables:
    mssh                  (nbpoints) float32 ...
    xtrack_cycle          (nbcycles) int32 157
    point                 (nbpoints) int32 ...
    time                  (nbpoints, nbcycles) object ...
    sla                   (nbpoints, nbcycles) float32 ...
    tide                  (nbpoints, nbcycles) float32 ...
    dac                   (nbpoints, nbcycles) float32 ...
    dist_to_coast_gshhs   (nbpoints) float64 ...
    dist_to_coast_stumpf  (nbpoints) float64 ...
    mdt_cnes_cls_18       (nbpoints) float64 ...
Attributes:
    title:        CTOH Along track Sea Level Anomalies
    institution:  CTOH/LEGOS, Toulouse Univ., CNRS, IRD, CNES, UPS, France
    Conventions:  CF-1.6
    history:      creation: 2020/09/10
    contact:      ctoh_products@legos.obs-mip.fr  http://ctoh.legos.obs-mip.fr
    version:      Version X-TRACK: 1.03.post18 - Version mercurial: hgcad1f04...
    doi:          10.6096/CTOH_X-TRACK_2017_02
    reference:    Birol, F. et al. “Coastal Applications from Nadir Altimetry...
    pass_number:  228
    handbook:     https://www.aviso.altimetry.fr/fileadmin/documents/data/too...

## Calculate across-track geostrophic current

In [ ]:
#describing what is done

In [47]:
#initalise array
u_geo = np.nan * ds.sla.data
#run for each cycle
for i in enumerate(cycle_array):
    #select respective data slice
    sel_cycle = ds.isel(nbcycles = i[0])
    sel_point = ds.isel(nbcycles = 0)
    #calculate geostrophic speeds for the cycle (dynamic comp)
    u_geo[:, i[0]] = geost1D(sel_cycle.lon.data, sel_cycle.lat.data, 1, sel_cycle.sla.data)
    #create DataArray for time
    time_da = xr.DataArray(data = ds.time.data, 
                           dims = ["loc", "cycle"], 
                           coords = dict(lat = ("loc", ds.lat.data),
                                         lon = ("loc", ds.lon.data), 
                                         cycle = ("cycle", ds.cycles.data.astype('int'))))
#convert to DataArray
u_geo = xr.DataArray(data = u_geo, dims = ["loc", "cycle"], coords = dict(lat = ("loc", ds.lat.data),
                                                                   lon = ("loc", ds.lon.data),
                                                                   cycle = ("cycle", ds.cycles.data.astype('int'))))
#calculate mean geostrophic speeds (mean comp) and convert to DataArray
u_mean = geost1D(sel_point.lon.data, sel_point.lat.data, 1, sel_point.mdt_cnes_cls_18.data)
u_mean = xr.DataArray(data = u_mean, dims = ["loc"], coords = dict(lat = ("loc", ds.lat.data),
                                                                  lon = ("loc", ds.lon.data)))
#calculate total geostrophic speed
u_total = u_geo + u_mean

#calculate zonal and meridional component

In [48]:
#checks
len(ds.sla.data[~np.isnan(ds.sla.data)])
len(u_total.data[~np.isnan(u_total.data)])

43

In [49]:
#create a dataset
geostrophic = xr.Dataset(data_vars = {"u_total": u_total, "u_geo": u_geo, "time": time_da, 
                                      "u_mean": u_mean})
geostrophic

<xarray.Dataset>
Dimensions:  (loc: 49, cycle: 1)
Coordinates:
    lat      (loc) float32 36.54 36.58 36.63 36.68 ... 38.75 38.8 38.85 38.89
    lon      (loc) float32 -73.33 -73.36 -73.39 -73.42 ... -74.83 -74.86 -74.88
  * cycle    (cycle) int64 157
Dimensions without coordinates: loc
Data variables:
    u_total  (loc, cycle) float64 nan 0.06018 0.7019 0.3431 ... nan nan nan nan
    u_geo    (loc, cycle) float32 nan -0.79 -0.06426 -0.3428 ... nan nan nan nan
    time     (loc, cycle) object 1996-12-26 20:44:10.871808 ... 1996-12-26 20...
    u_mean   (loc) float64 nan 0.8502 0.7662 0.6859 ... -0.1658 -0.1445 nan

## Save the file

In [50]:
geostrophic.to_netcdf("./data/alt_ugeo.nc")